1. dowload link in json file and then load into excel

In [10]:
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd
import json
import random

def get_headers():
    user_agent_list = [
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1.1 Safari/605.1.15',
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:77.0) Gecko/20100101 Firefox/77.0',
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36',
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:77.0) Gecko/20100101 Firefox/77.0',
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36',
    ]

    user_agent = random.choice(user_agent_list)

    # Set the headers
    headers = {'User-Agent': user_agent}
    return headers

# Prompt the user for input
query = "PFOA+removal+in+water"
pages = 1

data_15 = []
for i in range(0, pages * 10 + 1, 10):
    current_url = f"https://scholar.google.com/scholar?start={i}&q={query}&hl=en&as_sdt=0%2C34&as_vis=1&as_ylo=2020&as_yhi=2020"
    print(current_url)
    headers = get_headers()
    print("Going to hit %s" % current_url)
    res = requests.get(current_url, headers=headers)
    if res.status_code != 200:
        print(res.status_code, res.text)
    else:
        print(res.status_code, res.text)
    
    main_soup = BeautifulSoup(res.text, "html.parser")
    divs = main_soup.find_all("div", class_="gs_r gs_or gs_scl")

    for div in divs:
        temp = {}
        h3 = div.find("h3", class_="gs_rt")
        temp["Link"] = h3.find("a")["href"]
        temp["Heading"] = h3.find("a").get_text(strip=True)
        temp["Authors"] = div.find("div", class_="gs_a").get_text(strip=True)
        print(temp["Link"])
        try:
            res_link = requests.get(temp["Link"], headers=headers)
            soup_link = BeautifulSoup(res_link.text, "html.parser")
            if "sciencedirect" in temp["Link"]:
                temp["Abstract"] = soup_link.find("div", class_="abstract author").find("div").get_text(strip=True)
            elif "acm" in temp["Link"]:
                temp["Abstract"] = soup_link.find("div", class_="abstractSection abstractInFull").get_text(strip=True)
        except Exception as e:
            print(f"Error fetching link {temp['Link']}: {e}")
            pass

        if isinstance(temp, dict):
            if temp not in data_15:
                data_15.append(temp)
        else:
            print(f"The {temp} is not a dictionary.")
            
    print(data_15)
    
    with open(f"data_15.json", "w") as f:
        json.dump(data_15, f, indent=4)

    time.sleep(random.randint(45, 60))

# Save data to Excel file
df = pd.DataFrame(data_15)
df.to_excel("data_url_15.xlsx", index=False)

https://scholar.google.com/scholar?start=0&q=PFOA+removal+in+water&hl=en&as_sdt=0%2C34&as_vis=1&as_ylo=2020&as_yhi=2020
Going to hit https://scholar.google.com/scholar?start=0&q=PFOA+removal+in+water&hl=en&as_sdt=0%2C34&as_vis=1&as_ylo=2020&as_yhi=2020
429 <!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN">
<html>
<head><meta http-equiv="content-type" content="text/html; charset=utf-8"><meta name="viewport" content="initial-scale=1"><title>https://scholar.google.com/scholar?start=0&amp;q=PFOA+removal+in+water&amp;hl=en&amp;as_sdt=0%2C34&amp;as_vis=1&amp;as_ylo=2020&amp;as_yhi=2020</title></head>
<body style="font-family: arial, sans-serif; background-color: #fff; color: #000; padding:20px; font-size:18px; overscroll-behavior:contain;" onload="e=document.getElementById('captcha');if(e){e.focus();} if(solveSimpleChallenge) {solveSimpleChallenge(,);}">
<div style="max-width:400px;">
<hr noshade size="1" style="color:#ccc; background-color:#ccc;"><br>
<form id="captcha-form" ac

2. Retreive abstract from the downloaded link.

In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import pandas as pd
import random
import time
import json

def get_headers():
    user_agent_list = [
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1.1 Safari/605.1.15',
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:77.0) Gecko/20100101 Firefox/77.0',
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36',
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:77.0) Gecko/20100101 Firefox/77.0',
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36',
    ]

    user_agent = random.choice(user_agent_list)

    # Set the headers
    headers = {'User-Agent': user_agent}
    return headers

def get_article(url):
    try:
        # Set up Chrome options for headless browsing
        chrome_options = Options()
        chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36")
        chrome_options.add_argument("--headless")

        # Create a new instance of the Chrome driver
        browser = webdriver.Chrome(options=chrome_options)

        # Navigate to the page
        browser.get(url)

        # Sleep for a random duration between 300 and 600 seconds
        time.sleep(random.randint(3, 10))

        # Get HTML source
        html = browser.page_source
        # Close the browser
        browser.quit()

        soup = BeautifulSoup(html, 'html.parser')

        # Try finding abstract in ('div', class_='abstract author')
        abstract_div = soup.find('div', class_='abstract author')
        abstract_paragraph = abstract_div.find('p') if abstract_div else None

        # If not found, try finding in ('div', class_='abstract')
        if not abstract_paragraph:
            abstract_div = soup.find('div', class_='abstract')
            abstract_paragraph = abstract_div.find('p') if abstract_div else None

        abstract_text = abstract_paragraph.get_text(strip=True) if abstract_paragraph else ""



        if abstract_text:
            result = abstract_text
            return result

    except Exception as e:
        print(e)

# Rest of the code remains unchanged
# Rest of the code remains unchanged
def main():
    # Read the Excel file
    df = pd.read_excel("data_url.xlsx")

    # Create a new column for Abstract
    df['Abstract'] = df['Link'].apply(get_article)

    # Save the updated DataFrame to a new Excel file
    df.to_excel("abstracts-contained.xlsx", index=False)

if __name__ == "__main__":
    main()